## Иерархическая индексация (heirarchical indexing)
    Многомерные данные могут храниться путем использования объектов Panel - 3D, и Panel4D - для 4D или
    Иерархической индексации (или мультииндекс multi-indexing) для N-мерных структур.

In [1]:
import numpy as np
import pandas as pd

### Мультииндексированный объект Series

In [2]:
index = [
    ('c', 2000), ('c', 2010),
    ('n', 2000), ('n', 2010),
    ('t', 2000), ('t', 2010)
]
populations = [3387, 3725,
              1897, 1937,
              2085, 2514]

In [3]:
pop = pd.Series(populations, index=index)

In [4]:
pop
# это еще не мультииндекс, это картежи-ключи

(c, 2000)    3387
(c, 2010)    3725
(n, 2000)    1897
(n, 2010)    1937
(t, 2000)    2085
(t, 2010)    2514
dtype: int64

In [5]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('c', 2000),
            ('c', 2010),
            ('n', 2000),
            ('n', 2010),
            ('t', 2000),
            ('t', 2010)],
           )

In [6]:
pop = pop.reindex(index)
# вот это мультииндекс

In [7]:
pop

c  2000    3387
   2010    3725
n  2000    1897
   2010    1937
t  2000    2085
   2010    2514
dtype: int64

In [8]:
# мультииндекс любит срезы:
pop[:, 2010]

c    3725
n    1937
t    2514
dtype: int64

#### Мультииндекс как доп. измерение:

In [9]:
# мультииндексный Series м.б. преобразован в обычный DataFrame
pop_df = pop.unstack()

In [10]:
pop_df

,2000,2010
c,3387,3725
n,1897,1937
t,2085,2514


In [11]:
# и наоборот, DataFrame м.б. превращен в мультииндексный Series
pop_df.stack()

c  2000    3387
   2010    3725
n  2000    1897
   2010    1937
t  2000    2085
   2010    2514
dtype: int64

In [12]:
# можно добавать доп столбец
pop_df = pd.DataFrame({'total': pop,
                      'under18': [926, 928,
                                 468, 431,
                                 590, 687]})
pop_df

total  under18
c 2000   3387      926
  2010   3725      928
n 2000   1897      468
  2010   1937      431
t 2000   2085      590
  2010   2514      687

In [13]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18
# сакой способ позволяет легко манипулировать многомерными данными

c  2000    0.273398
   2010    0.249128
n  2000    0.246705
   2010    0.222509
t  2000    0.282974
   2010    0.273270
dtype: float64

In [14]:
f_u18.unstack()

,2000,2010
c,0.273398,0.249128
n,0.246705,0.222509
t,0.282974,0.273270


### Методы создания мультииндексов:
    Наиболее простой метод создания мультииндекса - передать в конструктор список из двух и более индексных массивов (стр. 168)

In [15]:
df = pd.DataFrame(np.random.rand(4,2),
                 index=[['a', 'a', 'b', 'b'],
                         [1, 2, 1, 2]],
                 columns=['data1', 'data2'])
df

data1     data2
a 1  0.272002  0.711487
  2  0.879524  0.594409
b 1  0.216776  0.009556
  2  0.562513  0.208733

In [16]:
# Если передавать словарь, с кортежами в качестве ключей Pandas автоматически распознает мультииндекс:
data = {
    ('c', 2000): 3387,
    ('c', 2010): 3725,
    ('n', 2000): 2085,
    ('n', 2010): 2514,
    ('t', 2000): 1897,
    ('t', 2010): 1937,
}

In [17]:
pd.Series(data)

c  2000    3387
   2010    3725
n  2000    2085
   2010    2514
t  2000    1897
   2010    1937
dtype: int64

### Явные конструкторы для MultiIndex
    pd.MultiIndex.методы-конструкторы класса

In [18]:
# из списка массивов
pd.MultiIndex.from_arrays([
    ['a', 'a', 'b', 'b'],
    [1, 2, 1, 2]
])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [19]:
# из списка кортежей
pd.MultiIndex.from_tuples([
    ('a', 1),
    ('a', 2),
    ('b', 1),
    ('b', 2)
])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

##### из декартова произведения обычных индексов:

In [20]:
pd.MultiIndex.from_product([
    ['a', 'b'],
    [1, 2]
])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [25]:
# с помощью внутреннего представления MultyIndex, передав:
# Levels - список списков значений для каждого уровня и
# codes - список списков меток
pd.MultiIndex(
    levels=[['a', 'b'], [1, 2]],
    codes=[[0, 0, 1, 1], [0, 1, 0, 1]]
)

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [22]:
pd.MultiIndex?
# pd.MultiIndex(
#     levels=None,
#     codes=None,
#     sortorder=None,
#     names=None,
#     dtype=None,
#     copy=False,
#     name=None,
#     verify_integrity: 'bool' = True,
# )

#### Названия уровней мультииндексов
    Иногда бывает удобно давать названия уровням мультииндексов

In [26]:
pop

c  2000    3387
   2010    3725
n  2000    1897
   2010    1937
t  2000    2085
   2010    2514
dtype: int64

In [27]:
pop.index.names = ['state', 'year']

In [28]:
pop

state  year
c      2000    3387
       2010    3725
n      2000    1897
       2010    1937
t      2000    2085
       2010    2514
dtype: int64

#### Мультииндекс для столбцов:
    В объектах DataFrame строки и столбцы полностью симметричны, и у строк и у столбцов может быть несколько уровней индексов

In [29]:
index = pd.MultiIndex.from_product(
    [[2013, 2014], [1, 2]],
    names=['year', 'visit']
)
columns = pd.MultiIndex.from_product(
    [['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
    names=['subject', 'type']
)

In [30]:
# создаем имитационные данные
data = np.round(np.random.randn(4, 6), 1)

In [31]:
data

array([[-0.4,  0.6, -0.4,  1.5, -1. ,  0.1],
       [-0.1, -0.7,  1.6, -1.3, -1. , -1. ],
       [-2.5,  0.8, -1.7,  0. ,  0.7, -0.5],
       [-1.9,  0.8,  0.2, -1.5, -0.8, -1.1]])

In [32]:
data[:, ::2] *= 10

In [33]:
data += 37

In [34]:
data

array([[33. , 37.6, 33. , 38.5, 27. , 37.1],
       [36. , 36.3, 53. , 35.7, 27. , 36. ],
       [12. , 37.8, 20. , 37. , 44. , 36.5],
       [18. , 37.8, 39. , 35.5, 29. , 35.9]])

In [35]:
helth_data = pd.DataFrame(data, index=index, columns=columns)
helth_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      33.0  37.6  33.0  38.5  27.0  37.1
     2      36.0  36.3  53.0  35.7  27.0  36.0
2014 1      12.0  37.8  20.0  37.0  44.0  36.5
     2      18.0  37.8  39.0  35.5  29.0  35.9

###### по сути это четырехмерные данные с измерениями:
- субъект
- измеряемый параметр type
- год
- номер посещения

In [37]:
helth_data['Guido']

type          HR  Temp
year visit            
2013 1      33.0  38.5
     2      53.0  35.7
2014 1      20.0  37.0
     2      39.0  35.5

In [ ]:
# это удобно для сложных записей, содержащих несколько маркированных неоднократно измеряемых параметров для многих субъектов

### Индексация и срезы по мультииндексу
    Объект MultiIndex сконструирован так, чтобы индексация и срезы по мультииндексу были интуитивно понятны, особенно если думать об индексации как о дополнительных измерениях.

#### Мультииндексация объектов Series:

In [38]:
pop

state  year
c      2000    3387
       2010    3725
n      2000    1897
       2010    1937
t      2000    2085
       2010    2514
dtype: int64

In [39]:
pop['c', 2000]

3387

In [40]:
# частичная индексация (partial indexing):
pop['c']

year
2000    3387
2010    3725
dtype: int64

In [41]:
# частичные срезы:
pop['c':'n']

state  year
c      2000    3387
       2010    3725
n      2000    1897
       2010    1937
dtype: int64

In [42]:
pop.loc['c':'n']

state  year
c      2000    3387
       2010    3725
n      2000    1897
       2010    1937
dtype: int64

In [44]:
# частичная индексация по нижним уровням:
pop[:, 2000]

state
c    3387
n    1897
t    2085
dtype: int64

In [46]:
# выборка данных на основе булевой маски:
pop[pop > 2500]

state  year
c      2000    3387
       2010    3725
t      2010    2514
dtype: int64

In [48]:
# выборка на основе прихотливой индексации:
pop[['c', 't']]

state  year
c      2000    3387
       2010    3725
t      2000    2085
       2010    2514
dtype: int64

#### Мультииндексация объектов DataFrame:

In [49]:
helth_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      33.0  37.6  33.0  38.5  27.0  37.1
     2      36.0  36.3  53.0  35.7  27.0  36.0
2014 1      12.0  37.8  20.0  37.0  44.0  36.5
     2      18.0  37.8  39.0  35.5  29.0  35.9

###### В объектах DataFrame основными являются столбцы

In [50]:
helth_data['Guido', 'HR']

year  visit
2013  1        33.0
      2        53.0
2014  1        20.0
      2        39.0
Name: (Guido, HR), dtype: float64

In [58]:
helth_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      33.0  37.6
     2      36.0  36.3

In [59]:
helth_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        33.0
      2        36.0
2014  1        12.0
      2        18.0
Name: (Bob, HR), dtype: float64

In [60]:
# доступ как в матрице
helth_data.loc[:, 'Bob']

type          HR  Temp
year visit            
2013 1      33.0  37.6
     2      36.0  36.3
2014 1      12.0  37.8
     2      18.0  37.8

In [62]:
# helth_data['Guido', 2013] - приведет к ошибке
# helth_data['Guido'][2013] - приведет к ошибке

In [63]:
helth_data.loc[(2013, 2), ('Bob', 'HR')]

36.0

In [64]:
# НО! срезы в виде кортежей могут привести к ошибке
helth_data.loc[(:, 2), ('Bob', 'HR')]

SyntaxError: invalid syntax (Temp/ipykernel_9536/3397362960.py, line 2)

In [65]:
# для работы со срезами в виде кортежей необходимо сформировать срез явным образом 
idx = pd.IndexSlice
helth_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,33.0,33.0,27.0
2014,1,12.0,20.0,44.0


### Перегруппировка мультииндексов
    Один из ключей к эффективной работе с данными умение эффективно их преобразовывать ради удобства вычислений.
    stack() unstack() и др.

#### Отсортированные и неотсортированные индексы
##### Большинство операций срезов с мультииндексами завершиться ошибкой, если индекс не отсортирован!!!

In [66]:
index = pd.MultiIndex.from_product(
    [['a', 'c', 'b'], [1, 2]]
)
index

MultiIndex([('a', 1),
            ('a', 2),
            ('c', 1),
            ('c', 2),
            ('b', 1),
            ('b', 2)],
           )

In [67]:
# index можно передавать как позиционный параметр
data = pd.Series(np.random.rand(6), index)

In [68]:
data

a  1    0.080243
   2    0.144768
c  1    0.607155
   2    0.371881
b  1    0.900655
   2    0.828185
dtype: float64

In [70]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)
# возникла ошибка, по причине того, что индекс не отсортирован

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


In [71]:
data = data.sort_index()

In [72]:
data

a  1    0.080243
   2    0.144768
b  1    0.900655
   2    0.828185
c  1    0.607155
   2    0.371881
dtype: float64

In [73]:
data['a':'b']

a  1    0.080243
   2    0.144768
b  1    0.900655
   2    0.828185
dtype: float64

#### Выполнение операции stack() unstack() над индексами
    Существует возможность преобразовать набор данных из вертикального мультииндекса в простое двумерное представление, при необходимости указывая требуемый уровнь.

In [74]:
pop

state  year
c      2000    3387
       2010    3725
n      2000    1897
       2010    1937
t      2000    2085
       2010    2514
dtype: int64

In [80]:
popa = pop.unstack()
popa
# level=1 преобразует 1 уровень индекса в имена столбцов, по умолчанию level=1

year,2000,2010
state,,
c,3387,3725
n,1897,1937
t,2085,2514


In [77]:
pop.unstack(level=0)
# level=0 преобразует 0 уровень индекса в имена столбцов

state,c,n,t
year,,,
2000,3387,1897,2085
2010,3725,1937,2514


In [78]:
pop.unstack(level=1)
# level=1 преобразует 1 уровень индекса в имена столбцов

year,2000,2010
state,,
c,3387,3725
n,1897,1937
t,2085,2514


In [81]:
popa.stack()

state  year
c      2000    3387
       2010    3725
n      2000    1897
       2010    1937
t      2000    2085
       2010    2514
dtype: int64

#### Создание и перестройка индексов
    Преобразование метки индекса в столбцы с помощью метода reset_index

In [86]:
pop_flat = pop.reset_index(name='population')

In [87]:
pop_flat
# часто реальные данные имеют подобный вид:

,state,year,population
0,c,2000,3387
1,c,2010,3725
2,n,2000,1897
3,n,2010,1937
4,t,2000,2085
5,t,2010,2514


# САМЫЙ УДОБНЫЙ ПАТТЕРН ДЛЯ РАБОТЫ С РЕАЛЬНЫМИ ДАННЫМИ:

In [88]:
# поэтому удобно создать MultiIndex из значений столбцов
pop_flat.set_index(['state', 'year'])

population
state year            
c     2000        3387
      2010        3725
n     2000        1897
      2010        1937
t     2000        2085
      2010        2514

### Агрегирование по мультииндексам

In [89]:
helth_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      33.0  37.6  33.0  38.5  27.0  37.1
     2      36.0  36.3  53.0  35.7  27.0  36.0
2014 1      12.0  37.8  20.0  37.0  44.0  36.5
     2      18.0  37.8  39.0  35.5  29.0  35.9

In [90]:
# вычисление среднего значения
data_mean = helth_data.mean(level='year')

C:\Users\Admin\AppData\Local\Temp/ipykernel_9536/3790674258.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  data_mean = helth_data.mean(level='year')


In [91]:
data_mean

subject   Bob        Guido          Sue       
type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     34.5  36.95  43.0  37.10  27.0  36.55
2014     15.0  37.80  29.5  36.25  36.5  36.20

In [93]:
# новый синтаксис для агрегаций
data_mean_f = helth_data.groupby('year').mean()
data_mean_f

subject   Bob        Guido          Sue       
type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     34.5  36.95  43.0  37.10  27.0  36.55
2014     15.0  37.80  29.5  36.25  36.5  36.20

In [95]:
# далее, воспользовавшись axis можно получить среднее значение по уровням по столбцам
data_mean_f.groupby(axis=1, level='type').mean()

type,HR,Temp
year,,
2013,34.833333,36.866667
2014,27.000000,36.750000


In [96]:
data_mean_f.mean(axis=1, level='type')

C:\Users\Admin\AppData\Local\Temp/ipykernel_9536/3676335015.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  data_mean_f.mean(axis=1, level='type')


type,HR,Temp
year,,
2013,34.833333,36.866667
2014,27.000000,36.750000
